In [ ]:
# Import packages 
import sys
sys.path.append('../Plotting/')
""" Now you can imprt modules in ../Plotting"""

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors

from scipy.io import FortranFile

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import ESMF as E

import importlib
import glob


import xyp_plot as xyp
import scripGen as SG

importlib.reload(SG)

In [ ]:
era5dir = "/glade/collections/rda/data/ds633.6/e5.oper.an.ml/"
wrkdir=era5dir+"202211/"
tfile = wrkdir + 'e5.oper.an.ml.0_5_0_0_0_t.regn320sc.2022110100_2022110105.nc'
spfile= wrkdir + 'e5.oper.an.ml.128_134_sp.regn320sc.2022110100_2022110105.nc'

In [ ]:
tdata  = xr.open_dataset( tfile )
spdata = xr.open_dataset( spfile )

In [ ]:
print( list( tdata.variables ) )

print( list( spdata.variables ) )


In [ ]:
print( np.shape(tdata['T']) , np.shape(tdata['longitude'])  , np.shape(tdata['latitude'])    )

In [ ]:
loco=E.LocStream(16,   coord_sys = E.CoordSys.SPH_DEG)

In [ ]:
dsc=SG.latlon_to_scrip( longitudes = tdata['longitude'].values , 
                      latitudes = tdata['latitude'].values )

In [ ]:
ERA5scripfile = 'ERA5_640x1280_scrip.nc'
dsc.to_netcdf( ERA5scripfile )

In [ ]:
dsX=SG.latlon_to_scrip( nx=1440 , ny=720 , lon0=0. ) 

In [ ]:
latitudes = tdata['latitude'].values
longitudes = tdata['longitude'].values

In [ ]:
latitudes[0:2]

print( np.min(latitudes) , 
np.max(latitudes) )


In [ ]:
plt.plot( latitudes[1:],latitudes[0:639]-latitudes[1:],'.' )
plt.ylim(.2809,.2811 )

In [ ]:
plt.plot( longitudes[1:],-longitudes[0:1279]+longitudes[1:],'.' )
plt.ylim(0.28124,0.28126)

In [ ]:
xc=dsc['grid_corner_lon'].values
yc=dsc['grid_corner_lat'].values

In [ ]:
print(np.max(xc) , np.min(xc) )
print(np.max(xc) - np.min(xc) )
print(np.max(yc) , np.min(yc) )
